In [1]:
import logging

import pandas as pd
import numpy as np

from copairs.map import run_pipeline, aggregate

In [2]:
logging.basicConfig(format='%(levelname)s:%(asctime)s:%(name)s:%(message)s')
logging.getLogger("copairs").setLevel(logging.INFO)

In [3]:
subset_corrected = pd.read_parquet("../3.correct/output/subset_profiles_mean_corrected_preprocessed.parquet")
subset_corrected

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,...,Nuclei_Texture_InfoMeas1_AGP_5_01_256,Nuclei_Texture_InfoMeas1_AGP_5_02_256,Nuclei_Texture_InfoMeas1_DNA_3_00_256,Nuclei_Texture_InfoMeas1_ER_3_00_256,Nuclei_Texture_InfoMeas1_Mito_3_03_256,Nuclei_Texture_InfoMeas1_Mito_5_03_256,Nuclei_Texture_InfoMeas1_RNA_3_00_256,Nuclei_Texture_InfoMeas2_AGP_3_00_256,Nuclei_Texture_InfoMeas2_ER_3_00_256,Nuclei_Texture_InfoMeas2_Mito_3_00_256
0,JCP2022_900006,ccsbBroad304_00008,ORF000425.1_TRC304.1,pLX_304,NM_001095.4,ASIC1,41,9606,acid sensing ion channel subunit 1,100.0,...,0.798157,0.146819,0.819576,-0.509169,0.472318,0.642681,0.066633,-0.881860,-0.350066,-1.073529
1,JCP2022_900176,ccsbBroad304_00189,ORF011559.1_TRC304.1,pLX_304,NM_000587.4,C7,730,9606,complement C7,100.0,...,-0.008716,-0.727536,0.174183,-1.533901,-0.491498,0.133856,-0.166967,-0.046219,0.143956,-0.708162
2,JCP2022_900198,ccsbBroad304_00212,ORF000189.1_TRC304.1,pLX_304,NM_001745.4,CAMLG,819,9606,calcium modulating ligand,100.0,...,0.092668,-0.227209,0.816750,-0.338482,0.779559,0.923186,0.256518,0.053587,-0.516253,-1.248180
3,JCP2022_900294,ccsbBroad304_00321,ORF012872.1_TRC304.1,pLX_304,NM_001830.4,CLCN4,1183,9606,chloride voltage-gated channel 4,100.0,...,0.276906,-0.083525,0.298071,-0.617495,-0.362687,-0.138488,0.048434,-0.356286,-0.416854,-0.335360
4,JCP2022_900295,ccsbBroad304_00322,ORF012041.1_TRC304.1,pLX_304,NM_000084.5,CLCN5,1184,9606,chloride voltage-gated channel 5,100.0,...,0.042108,-0.414073,0.105622,-1.180956,-1.491862,-1.289506,-0.893941,-0.129673,0.202546,0.657406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19139,JCP2022_915130,ccsbBroad304_99991,ORFC00003.1_TRC304.1,pLX_304,promegaLuc.1,LUCIFERASE,LUCIFERASE,CONTROL,promegaLuc,100.0,...,-2.129138,-3.538450,-0.520974,-2.842374,-2.340930,-0.125043,0.631924,1.786284,0.033267,-0.950689
19140,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,-1.647278,-2.680781,-0.061632,-2.122849,-1.650066,0.044767,0.398538,1.444183,-0.247909,-1.030216
19141,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,-2.226537,-3.621706,-0.492188,-2.546204,-2.909455,-0.634502,0.987795,2.092715,-0.198424,-0.264739
19142,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,-3.100481,-4.689853,-0.735725,-3.097747,-2.915065,-1.003161,1.391602,2.520438,-0.121587,-0.152909


In [4]:
metadata = subset_corrected.filter(regex="^Metadata_")
feature_values = subset_corrected.filter(regex="^(?!Metadata_)").values.shape

In [7]:
metadata

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,Metadata_Insert_Length,Metadata_pert_type,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_Plate_Map_Name
0,JCP2022_900006,ccsbBroad304_00008,ORF000425.1_TRC304.1,pLX_304,NM_001095.4,ASIC1,41,9606,acid sensing ion channel subunit 1,100.0,1584.0,trt,source_4,BR00126538,B21,OAB37.38.39.40.A
1,JCP2022_900176,ccsbBroad304_00189,ORF011559.1_TRC304.1,pLX_304,NM_000587.4,C7,730,9606,complement C7,100.0,2529.0,trt,source_4,BR00126538,I15,OAB37.38.39.40.A
2,JCP2022_900198,ccsbBroad304_00212,ORF000189.1_TRC304.1,pLX_304,NM_001745.4,CAMLG,819,9606,calcium modulating ligand,100.0,888.0,trt,source_4,BR00126538,K21,OAB37.38.39.40.A
3,JCP2022_900294,ccsbBroad304_00321,ORF012872.1_TRC304.1,pLX_304,NM_001830.4,CLCN4,1183,9606,chloride voltage-gated channel 4,100.0,2280.0,trt,source_4,BR00126538,L17,OAB37.38.39.40.A
4,JCP2022_900295,ccsbBroad304_00322,ORF012041.1_TRC304.1,pLX_304,NM_000084.5,CLCN5,1184,9606,chloride voltage-gated channel 5,100.0,2238.0,trt,source_4,BR00126538,L20,OAB37.38.39.40.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19139,JCP2022_915130,ccsbBroad304_99991,ORFC00003.1_TRC304.1,pLX_304,promegaLuc.1,LUCIFERASE,LUCIFERASE,CONTROL,promegaLuc,100.0,1650.0,negcon,source_4,BR00123791,M12,OAB41.OAC17.OAB78.79.A
19140,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,3204.0,negcon,source_4,BR00123791,F17,OAB41.OAC17.OAB78.79.A
19141,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,3204.0,negcon,source_4,BR00123791,I10,OAB41.OAC17.OAB78.79.A
19142,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,3204.0,negcon,source_4,BR00123791,K09,OAB41.OAC17.OAB78.79.A


In [5]:
pos_sameby = ["Metadata_Well"]
pos_diffby = ["Metadata_JCP2022"]

neg_sameby = ["Metadata_Vector"]
neg_diffby = ["Metadata_pert_type"]

null_size = 10000

In [6]:
result = run_pipeline(
    metadata,
    feature_values, 
    pos_sameby,
    pos_diffby,
    neg_sameby,
    neg_diffby,
    null_size
)

INFO:2023-05-01 15:17:57,822:copairs:Finding positive and negative pairs...
INFO:2023-05-01 15:18:07,344:copairs:Computing positive similarities...


  0%|          | 0/23 [00:00<?, ?it/s]

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
agg_result = aggregate(result, sameby=pos_sameby, threshold=0.05)
agg_result